In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=2ec53f93f31a277c0063afa6c6eaaeb540f6dd79b1d4bd2a1b72a78f64cd94d5
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=abfd8fd9d0e0ff85651d17d36d7f39853bd7df5f9fbca608018b828d9ce21b7b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [61]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashin_mnist=keras.datasets.fashion_mnist


In [ ]:
(train_images,train_lables),(test_images,test_lables)=fashin_mnist.load_data()

In [ ]:
train_images=train_images/255.0
test_images-test_images/255.0

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [83]:
def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                              activation='relu',
                              input_shape=(28,28,1)
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_2_filters',min_value=32,max_value=64,step=16),
                              kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                              activation='relu'
                              
                          ),
                          keras.layers.Flatten(),
                          keras.layers.Dense(
                              units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                              activation='relu'

                          ),
                          keras.layers.Dense(10,activation='softmax')#output layer
                          
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                          loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [76]:
from kerastuner import RandomSearch
from tensorflow.keras import layers
from kerastuner.engine.hyperparameters import HyperParameters

In [84]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5, directory='output',project_name="Mnist Fashion")

In [85]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 16m 56s]
val_accuracy: 0.9141666889190674

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 01h 35m 37s
INFO:tensorflow:Oracle triggered exit


In [87]:
model=tuner_search.get_best_models(num_models=1)[0]

In [88]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        51232     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1734768   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 1,787,770
Trainable params: 1,787,770
Non-trainable params: 0
_________________________________________________________________
